In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
pg = 'https://electproject.github.io/Early-Vote-2020G/index.html'
r = requests.Session().get(pg)
soup = BeautifulSoup(r.content, 'html.parser')

In [3]:
import re
state_url = re.compile(r'/[A-Z]{2}\.html')

urls = [a['href'] for a in soup.find_all('a', attrs={'href':state_url})]
urls

['https://electproject.github.io/Early-Vote-2020G/AK.html',
 'https://electproject.github.io/Early-Vote-2020G/AL.html',
 'https://electproject.github.io/Early-Vote-2020G/AR.html',
 'https://electproject.github.io/Early-Vote-2020G/AZ.html',
 'https://electproject.github.io/Early-Vote-2020G/CA.html',
 'https://electproject.github.io/Early-Vote-2020G/CO.html',
 'https://electproject.github.io/Early-Vote-2020G/CT.html',
 'https://electproject.github.io/Early-Vote-2020G/DC.html',
 'https://electproject.github.io/Early-Vote-2020G/DE.html',
 'https://electproject.github.io/Early-Vote-2020G/FL.html',
 'https://electproject.github.io/Early-Vote-2020G/GA.html',
 'https://electproject.github.io/Early-Vote-2020G/HI.html',
 'https://electproject.github.io/Early-Vote-2020G/IA.html',
 'https://electproject.github.io/Early-Vote-2020G/ID.html',
 'https://electproject.github.io/Early-Vote-2020G/IL.html',
 'https://electproject.github.io/Early-Vote-2020G/IN.html',
 'https://electproject.github.io/Early-V

In [4]:
def parse_page(soup):
    values_dict = {}    
    state = soup.find('h1', class_="title toc-ignore").text.split(" Early Voting Statistics")[0]
    values_dict['state'] = state
    for strong in soup.find_all('strong'):
        if "ballot request lead" in strong.parent.text:
            continue
        value = strong.text.replace(',', '')
        try:
            value = int(value)
            context_id = strong.parent.parent['id']
            values_dict[context_id] = value
        except ValueError: # string or percentage, not number of votes
            pass
    return values_dict

In [6]:
a_page = requests.Session().get(urls[0])
a_soup = BeautifulSoup(a_page.content, 'html.parser')
strongs = a_soup.find_all('strong')
strongs

[<strong>113,307</strong>, <strong>161,061</strong>]

In [7]:
[s.parent.text for s in strongs]

['Alaska voters have cast 113,307 mail ballots.',
 'Alaska voters have requested 161,061 mail ballots.']

In [8]:
parse_page(a_soup)

{'state': 'Alaska',
 'returned-mail-ballots': 113307,
 'mail-ballot-requests': 161061}

In [9]:
def make_soup(url: str) -> BeautifulSoup:
    print(f"Processing page at {url}")
    s = requests.Session()
    r = s.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [10]:
soups = [make_soup(url) for url in urls]

Processing page at https://electproject.github.io/Early-Vote-2020G/AK.html
Processing page at https://electproject.github.io/Early-Vote-2020G/AL.html
Processing page at https://electproject.github.io/Early-Vote-2020G/AR.html
Processing page at https://electproject.github.io/Early-Vote-2020G/AZ.html
Processing page at https://electproject.github.io/Early-Vote-2020G/CA.html
Processing page at https://electproject.github.io/Early-Vote-2020G/CO.html
Processing page at https://electproject.github.io/Early-Vote-2020G/CT.html
Processing page at https://electproject.github.io/Early-Vote-2020G/DC.html
Processing page at https://electproject.github.io/Early-Vote-2020G/DE.html
Processing page at https://electproject.github.io/Early-Vote-2020G/FL.html
Processing page at https://electproject.github.io/Early-Vote-2020G/GA.html
Processing page at https://electproject.github.io/Early-Vote-2020G/HI.html
Processing page at https://electproject.github.io/Early-Vote-2020G/IA.html
Processing page at https:

In [11]:
data_dict = [parse_page(soup) for soup in soups]

In [14]:
import json
json_data = [json.dumps(d, indent=4) for d in data_dict]

In [15]:
with open("voting_data.json", "w") as outfile:
    for part in json_data:
        outfile.write(part)